In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
train = pd.read_csv('data/train.csv')
train['date'] = pd.to_datetime(train['date'])
train

,apart_to_room,num_builds_live,mean_price,date,num_builds_series_live,room_three,id_house,med_price,room_four,room_one,...,number_total,room_two,vc_city_quadkey,healthcare_cnt,flats_cnt,beauty_cnt,shopping_cnt,build_year_median,lng,lat
0,0.0,1,27272.728125,2021-02-01,1,0.000000,4139,27272.728125,0.0,0.000000,...,1,1.000000,1,1.0,0.0,0.0,0.0,NaN,58.426689,79.093861
1,0.0,1,27272.728125,2021-03-01,1,0.000000,4139,27272.728125,0.0,0.000000,...,1,1.000000,1,1.0,0.0,0.0,0.0,NaN,58.426689,79.093861
2,0.0,1,52173.914062,2021-06-01,1,0.000000,7054,52173.914062,0.0,1.000000,...,1,0.000000,1,0.0,0.0,0.0,0.0,NaN,58.525566,79.002254
3,0.0,1,52173.914062,2021-07-01,1,0.000000,7054,52173.914062,0.0,1.000000,...,1,0.000000,1,0.0,0.0,0.0,0.0,NaN,58.525566,79.002254
4,0.0,1,43500.000000,2020-01-01,1,1.000000,6123,42857.142188,0.0,0.000000,...,3,0.000000,9,0.0,1.0,0.0,0.0,1981.5,59.107842,79.032814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150302,0.0,6,22816.119727,2022-01-01,1,0.062500,5668,22336.957031,0.0,0.000000,...,16,0.937500,23,3.0,0.0,0.0,0.0,1972.0,62.612480,77.772386
150303,0.0,6,22945.890000,2022-02-01,1,0.066667,5668,22500.000000,0.0,0.000000,...,15,0.933333,23,3.0,0.0,0.0,0.0,1972.0,62.612480,77.772386
150304,0.0,8,22566.019401,2022-03-01,1,0.055556,5668,22336.957031,0.0,0.055556,...,18,0.888889,23,3.0,0.0,0.0,0.0,1972.0,62.612480,77.772386
150305,0.0,9,22493.675699,2022-04-01,1,0.052632,5668,22173.914062,0.0,0.052632,...,19,0.894737,23,3.0,0.0,0.0,0.0,1972.0,62.612480,77.772386


In [4]:
train.columns

Index(['apart_to_room', 'num_builds_live', 'mean_price', 'date',
       'num_builds_series_live', 'room_three', 'id_house', 'med_price',
       'room_four', 'room_one', 'mean_area', 'room_zero', 'number_total',
       'room_two', 'vc_city_quadkey', 'healthcare_cnt', 'flats_cnt',
       'beauty_cnt', 'shopping_cnt', 'build_year_median', 'lng', 'lat'],
      dtype='object')

In [5]:
test = pd.read_csv('data/sample_submission.csv')
test['date'] = pd.to_datetime(test['id'].apply(lambda x: x.split('_')[0]))
test['id_house'] = test['id'].apply(lambda x: x.split('_')[1]).astype(int)
test

,id,target,date,id_house
0,2022-06-01_6123,0,2022-06-01,6123
1,2022-07-01_6123,0,2022-07-01,6123
2,2022-08-01_6123,0,2022-08-01,6123
3,2022-06-01_4104,0,2022-06-01,4104
4,2022-07-01_4104,0,2022-07-01,4104
...,...,...,...,...
15592,2022-07-01_6599,0,2022-07-01,6599
15593,2022-08-01_6599,0,2022-08-01,6599
15594,2022-06-01_5668,0,2022-06-01,5668
15595,2022-07-01_5668,0,2022-07-01,5668


In [6]:
train = train[train['id_house'].isin(test['id_house'])]
train

,apart_to_room,num_builds_live,mean_price,date,num_builds_series_live,room_three,id_house,med_price,room_four,room_one,...,number_total,room_two,vc_city_quadkey,healthcare_cnt,flats_cnt,beauty_cnt,shopping_cnt,build_year_median,lng,lat
4,0.0,1,43500.000000,2020-01-01,1,1.000000,6123,42857.142188,0.0,0.000000,...,3,0.000000,9,0.0,1.0,0.0,0.0,1981.5,59.107842,79.032814
5,0.0,1,43500.000000,2020-02-01,1,1.000000,6123,42857.142188,0.0,0.000000,...,3,0.000000,9,0.0,1.0,0.0,0.0,1981.5,59.107842,79.032814
6,0.0,1,43500.000000,2020-03-01,1,1.000000,6123,42857.142188,0.0,0.000000,...,3,0.000000,9,0.0,1.0,0.0,0.0,1981.5,59.107842,79.032814
7,0.0,1,43500.000000,2020-04-01,1,1.000000,6123,42857.142188,0.0,0.000000,...,3,0.000000,9,0.0,1.0,0.0,0.0,1981.5,59.107842,79.032814
8,0.0,1,43500.000000,2020-05-01,1,1.000000,6123,42857.142188,0.0,0.000000,...,3,0.000000,9,0.0,1.0,0.0,0.0,1981.5,59.107842,79.032814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150302,0.0,6,22816.119727,2022-01-01,1,0.062500,5668,22336.957031,0.0,0.000000,...,16,0.937500,23,3.0,0.0,0.0,0.0,1972.0,62.612480,77.772386
150303,0.0,6,22945.890000,2022-02-01,1,0.066667,5668,22500.000000,0.0,0.000000,...,15,0.933333,23,3.0,0.0,0.0,0.0,1972.0,62.612480,77.772386
150304,0.0,8,22566.019401,2022-03-01,1,0.055556,5668,22336.957031,0.0,0.055556,...,18,0.888889,23,3.0,0.0,0.0,0.0,1972.0,62.612480,77.772386
150305,0.0,9,22493.675699,2022-04-01,1,0.052632,5668,22173.914062,0.0,0.052632,...,19,0.894737,23,3.0,0.0,0.0,0.0,1972.0,62.612480,77.772386


In [7]:
train.groupby('id_house').std().sum() == 0

apart_to_room             False
num_builds_live           False
mean_price                False
date                      False
num_builds_series_live    False
room_three                False
med_price                 False
room_four                 False
room_one                  False
mean_area                 False
room_zero                 False
number_total              False
room_two                  False
vc_city_quadkey            True
healthcare_cnt             True
flats_cnt                  True
beauty_cnt                 True
shopping_cnt               True
build_year_median          True
lng                        True
lat                        True
dtype: bool

In [8]:
train.isna().mean()

apart_to_room             0.000000
num_builds_live           0.000000
mean_price                0.000000
date                      0.000000
num_builds_series_live    0.000000
room_three                0.000000
id_house                  0.000000
med_price                 0.000000
room_four                 0.000000
room_one                  0.000000
mean_area                 0.000000
room_zero                 0.000000
number_total              0.000000
room_two                  0.000000
vc_city_quadkey           0.000000
healthcare_cnt            0.008258
flats_cnt                 0.000000
beauty_cnt                0.008258
shopping_cnt              0.008258
build_year_median         0.022105
lng                       0.000000
lat                       0.000000
dtype: float64

In [9]:
train[~train['healthcare_cnt'].isna()]['mean_price'].mean(), train[train['healthcare_cnt'].isna()]['mean_price'].mean()

(142808.6158828121, 61013.25688439008)

In [10]:
train[~train['build_year_median'].isna()]['mean_price'].mean(), train[train['build_year_median'].isna()]['mean_price'].mean()

(143724.1389322686, 71749.27944236575)

In [11]:
train['mean_price'].mean()

142133.1318364882

In [12]:
train

,apart_to_room,num_builds_live,mean_price,date,num_builds_series_live,room_three,id_house,med_price,room_four,room_one,...,number_total,room_two,vc_city_quadkey,healthcare_cnt,flats_cnt,beauty_cnt,shopping_cnt,build_year_median,lng,lat
4,0.0,1,43500.000000,2020-01-01,1,1.000000,6123,42857.142188,0.0,0.000000,...,3,0.000000,9,0.0,1.0,0.0,0.0,1981.5,59.107842,79.032814
5,0.0,1,43500.000000,2020-02-01,1,1.000000,6123,42857.142188,0.0,0.000000,...,3,0.000000,9,0.0,1.0,0.0,0.0,1981.5,59.107842,79.032814
6,0.0,1,43500.000000,2020-03-01,1,1.000000,6123,42857.142188,0.0,0.000000,...,3,0.000000,9,0.0,1.0,0.0,0.0,1981.5,59.107842,79.032814
7,0.0,1,43500.000000,2020-04-01,1,1.000000,6123,42857.142188,0.0,0.000000,...,3,0.000000,9,0.0,1.0,0.0,0.0,1981.5,59.107842,79.032814
8,0.0,1,43500.000000,2020-05-01,1,1.000000,6123,42857.142188,0.0,0.000000,...,3,0.000000,9,0.0,1.0,0.0,0.0,1981.5,59.107842,79.032814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150302,0.0,6,22816.119727,2022-01-01,1,0.062500,5668,22336.957031,0.0,0.000000,...,16,0.937500,23,3.0,0.0,0.0,0.0,1972.0,62.612480,77.772386
150303,0.0,6,22945.890000,2022-02-01,1,0.066667,5668,22500.000000,0.0,0.000000,...,15,0.933333,23,3.0,0.0,0.0,0.0,1972.0,62.612480,77.772386
150304,0.0,8,22566.019401,2022-03-01,1,0.055556,5668,22336.957031,0.0,0.055556,...,18,0.888889,23,3.0,0.0,0.0,0.0,1972.0,62.612480,77.772386
150305,0.0,9,22493.675699,2022-04-01,1,0.052632,5668,22173.914062,0.0,0.052632,...,19,0.894737,23,3.0,0.0,0.0,0.0,1972.0,62.612480,77.772386


In [13]:
df = {'id_house': [], 'date': []}
ar = train['id_house'].sort_values().unique()
br = train['date'].sort_values().unique()
for a in ar:
    for b in br:
        df['id_house'].append(a)
        df['date'].append(b)
df = pd.DataFrame(df)


df = df.merge(train[['id_house', 'date', 'mean_price']], 'left', on=['id_house', 'date'])
sdf = df.copy()
sdf['date'] = sdf['date'].dt.to_period('M').astype(str)


df = df.merge(train.groupby('id_house')[['build_year_median']].first().reset_index(), 'left', on=['id_house'])


subset = ['build_year_median']

gbmp = df.groupby('id_house', group_keys=False)['mean_price']

df['mean_price_diff_rw_2_std'] = gbmp.apply(lambda x: x.diff().rolling(2).std())
subset.append('mean_price_diff_rw_2_std')
df['mean_price_lag_0'] = gbmp.apply(lambda x: x.shift(0))
subset.append('mean_price_lag_0')

for i in range(12):
    df[f'mean_price_diff_0_{i + 1}'] = gbmp.apply(lambda x: x.diff(periods=i + 1))
    subset.append(f'mean_price_diff_0_{i + 1}')
    if i > 0:
        df[f'mean_price_diff_{i}_{i + 1}'] = gbmp.apply(lambda x: x.diff().shift(i))
        subset.append(f'mean_price_diff_{i}_{i + 1}')

df['target'] = df.groupby('id_house')['mean_price'].diff().shift(-1) #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
df = df.dropna(subset=['target'])

In [14]:
s = []
bad = ['mean_price_diff_0_12', 'mean_price_diff_0_4', 'mean_price_diff_5_6',
       'mean_price_diff_2_3', 'mean_price_diff_0_6', 'mean_price_diff_0_3',
       'mean_price_diff_0_11']
for i in subset:
    if i not in bad:
        s.append(i)
subset = s

In [15]:
from sklearn.cluster import DBSCAN, KMeans
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import euclidean_distances

p = df[subset + ['id_house']].dropna()

n_components_pca = 8
pca_cols = [f'PCA_{i}' for i in range(n_components_pca)]
pca = PCA(n_components=n_components_pca)
pca.fit(p[subset])

pca_features = pd.DataFrame(pca.transform(p[subset]), 
                            index=p.index, 
                            columns=pca_cols)
pca_features['id_house'] = p['id_house']

by = pca_features.groupby('id_house').mean()
distance_matrix = pd.DataFrame(euclidean_distances(by.values), index=by.index, columns=by.index)

# clustering = DBSCAN(eps=10000, min_samples=5, metric="precomputed")
# clusters = pd.DataFrame({'id_house': distance_matrix.index, 'cluster': clustering.fit_predict(distance_matrix)})

n_clusters = 10  # Adjust the number of clusters as needed
clustering = KMeans(n_clusters=n_clusters, random_state=42)
clusters = pd.DataFrame({'id_house': distance_matrix.index, 
                         'cluster': clustering.fit_predict(distance_matrix).astype(int)})

clusters['cluster'].value_counts()

cluster
2    1565
6    1257
0     689
9     593
5     237
8      93
3      48
1      40
7      18
4       8
Name: count, dtype: int64

In [16]:
df = df.merge(clusters, 'left', 'id_house')
df['cluster'] = df['cluster'].fillna(-1).astype(int)#.map(clusters['cluster'].value_counts())
# subset.append('cluster')

In [17]:
df

,id_house,date,mean_price,build_year_median,mean_price_diff_rw_2_std,mean_price_lag_0,mean_price_diff_0_1,mean_price_diff_0_2,mean_price_diff_1_2,mean_price_diff_0_3,...,mean_price_diff_0_9,mean_price_diff_8_9,mean_price_diff_0_10,mean_price_diff_9_10,mean_price_diff_0_11,mean_price_diff_10_11,mean_price_diff_0_12,mean_price_diff_11_12,target,cluster
0,1,2020-01-01,113026.606534,2013.0,NaN,113026.606534,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4499.154034,6
1,1,2020-02-01,108527.452500,2013.0,NaN,108527.452500,-4499.154034,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,6
2,1,2020-03-01,108527.452500,2013.0,3181.382327,108527.452500,0.000000,-4499.154034,-4499.154034,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,6
3,1,2020-04-01,108527.452500,2013.0,0.000000,108527.452500,0.000000,0.000000,0.000000,-4499.154034,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,6
4,1,2020-05-01,108527.452500,2013.0,0.000000,108527.452500,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129905,7095,2021-12-01,121083.147869,2014.0,243.657895,121083.147869,2283.658619,4911.901538,2628.242918,5729.848651,...,8525.218641,190.997546,9378.102334,852.883692,12119.110890,2741.008557,11341.951655,-777.159235,232.399432,6
129906,7095,2022-01-01,121315.547301,2014.0,1450.459281,121315.547301,232.399432,2516.058051,2283.658619,5144.300970,...,8566.620527,784.198226,8757.618073,190.997546,9610.501765,852.883692,12351.510322,2741.008557,-385.707197,6
129907,7095,2022-02-01,120929.840104,2014.0,437.067389,120929.840104,-385.707197,-153.307765,232.399432,2130.350854,...,7396.715104,-3605.659778,8180.913331,784.198226,8371.910876,190.997546,9224.794568,852.883692,692.345751,6
129908,7095,2022-03-01,121622.185855,2014.0,762.298550,121622.185855,692.345751,306.638554,-385.707197,539.037986,...,11694.720633,2792.121653,8089.060855,-3605.659778,8873.259082,784.198226,9064.256627,190.997546,1430.244874,6


In [18]:
all_months = sorted(df['date'].unique())
N_valid_months = 3
valid_threshold = all_months[-N_valid_months]  # первая из последних 3
train_data = df[df['date'] < valid_threshold].sample(frac=1, random_state=42)
valid_data = df[df['date'] >= valid_threshold]

In [19]:
feature_cols = subset
cat_cols = ['id_house'] #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

X_train = train_data[train_data['target'] != 0][feature_cols + cat_cols]
y_train = train_data[train_data['target'] != 0]['target']

X_valid = valid_data[valid_data['target'] != 0][feature_cols + cat_cols]
y_valid = valid_data[valid_data['target'] != 0]['target']

cat_features_indices = [X_train.columns.get_loc(c) for c in cat_cols]

In [20]:
# X_train['random_feature'] = np.random.rand(len(X_train))
# X_valid['random_feature'] = np.random.rand(len(X_valid))

In [21]:
X_train

,build_year_median,mean_price_diff_rw_2_std,mean_price_lag_0,mean_price_diff_0_1,mean_price_diff_0_2,mean_price_diff_1_2,mean_price_diff_3_4,mean_price_diff_0_5,mean_price_diff_4_5,mean_price_diff_0_7,mean_price_diff_6_7,mean_price_diff_0_8,mean_price_diff_7_8,mean_price_diff_0_9,mean_price_diff_8_9,mean_price_diff_0_10,mean_price_diff_9_10,mean_price_diff_10_11,mean_price_diff_11_12,id_house
54787,1994.0,461.575136,51583.607520,417.726445,182.687074,-235.039372,-278.323378,19.873739,44.394717,904.741806,685.374457,1150.688098,245.946292,530.530431,-620.157667,400.208772,-130.321658,926.622685,-234.800934,3013
2108,2010.0,707.950392,76562.099682,-27.484470,946.224107,973.708576,202.097717,1071.364690,-587.772887,-4170.910880,-5148.284624,-4901.191279,-730.280400,-4547.758437,353.432842,-4386.181418,161.577020,-976.958386,-755.070124,115
124627,1998.0,611.985321,104199.876011,1086.104779,3037.687500,1951.582721,122.795371,838.948199,-1866.191250,1011.481434,69.780683,1527.875816,516.394382,NaN,NaN,NaN,NaN,NaN,NaN,6807
56335,1993.0,888.041043,48049.364844,-2940.042578,-4624.205469,-1684.162891,4289.357812,-863.771875,-1138.244531,-3205.964453,247.900391,-4244.607344,-1038.642891,-5470.465625,-1225.858281,-5470.465625,0.000000,986.815554,NaN,3095
88130,1988.0,187.716981,88267.721259,35.741370,336.954640,301.213270,-1387.620430,-142.368692,1159.356420,-551.771901,33.110202,-1402.639149,-850.867247,-1624.500371,-221.861223,-2566.261683,-941.761312,NaN,NaN,4841
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62101,1983.0,401.908267,186199.249432,-353.028888,-1274.441899,-921.413010,-186.143534,-5003.768425,180.950893,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3417
86945,2008.0,4704.830539,139089.236719,0.000000,-6653.635156,-6653.635156,0.000000,-6653.635156,0.000000,-8182.605469,0.000000,-12064.966406,-3882.360937,-14183.207812,-2118.241406,-14183.207812,0.000000,0.000000,0.000000,4779
124741,1978.0,4067.384837,117316.890234,-1454.539729,2843.071342,4297.611071,1057.059871,4714.677133,812.028529,6417.705636,-298.910353,8006.158033,1588.452397,4466.739609,-3539.418424,2168.736541,-2298.003068,3867.020053,-4.151360,6811
117311,1969.5,87.456659,49595.756813,-352.996911,-582.311429,-229.314518,-1330.093453,-2276.178898,-581.600731,-1440.266391,95.612155,-1064.056112,376.210279,-1064.056112,0.000000,-1496.192081,-432.135969,48.817115,-458.063897,6428


In [22]:
X_train.columns, len(X_train.columns)

(Index(['build_year_median', 'mean_price_diff_rw_2_std', 'mean_price_lag_0',
        'mean_price_diff_0_1', 'mean_price_diff_0_2', 'mean_price_diff_1_2',
        'mean_price_diff_3_4', 'mean_price_diff_0_5', 'mean_price_diff_4_5',
        'mean_price_diff_0_7', 'mean_price_diff_6_7', 'mean_price_diff_0_8',
        'mean_price_diff_7_8', 'mean_price_diff_0_9', 'mean_price_diff_8_9',
        'mean_price_diff_0_10', 'mean_price_diff_9_10', 'mean_price_diff_10_11',
        'mean_price_diff_11_12', 'id_house'],
       dtype='object'),
 20)

In [23]:
from tabnanny import verbose


catboost_model = CatBoostRegressor(
    iterations=3700,
    learning_rate=0.0005,
    depth=12,
    subsample=0.8,
    colsample_bylevel=0.9,
    l2_leaf_reg=0.9,
    random_seed=42,
    cat_features=cat_features_indices,
    loss_function='MAE',
    verbose=0
)

lgbm_model = LGBMRegressor(
    n_estimators=4200,
    learning_rate=0.001,
    num_leaves=128,
    subsample=0.8,
    colsample_bytree=0.9,
    reg_lambda=0.9,
    random_state=42,
    verbose=-1
)

xgb_model = XGBRegressor(
    n_estimators=3700,
    learning_rate=0.0005,
    max_depth=12,
    subsample=0.8,
    colsample_bytree=0.9,
    reg_lambda=0.9,
    random_state=42
)

rf_model = RandomForestRegressor(
    n_estimators=500,
    max_depth=10,
    random_state=42
)

model = StackingRegressor(
    estimators=[
        ('catboost', catboost_model),
        ('lgbm', lgbm_model),
        ('xgb', xgb_model)
    ],
    final_estimator=rf_model,
    passthrough=True
)

model.fit(X_train, y_train)

AttributeError: 'super' object has no attribute '__sklearn_tags__'

In [ ]:
pd.DataFrame(model.feature_importances_, index=model.feature_names_, columns=['importance']).sort_values(by='importance', ascending=False)

In [ ]:
y_pred_valid = model.predict(X_valid)
mae_valid = mean_absolute_error(y_valid, y_pred_valid)
print("MAE (valid):  ", mae_valid)
print("Median Error: ", (y_valid - y_pred_valid).median())

In [35]:
def predict_next_month(df, model):
    df = df.copy()
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m')
    df['date_pred'] = (df['date'] + pd.offsets.MonthBegin(1)).dt.to_period('M').astype(str)
    df = df.sort_values(by=['id_house', 'date'])

    df = df.merge(train.groupby('id_house')[['build_year_median']].first().reset_index(), 'left', on=['id_house'])

    gbmp = df.groupby('id_house', group_keys=False)['mean_price']

    df['mean_price_diff_rw_2_std'] = gbmp.apply(lambda x: x.diff().rolling(2).std())
    df['mean_price_lag_0'] = gbmp.apply(lambda x: x.shift(0))

    for i in range(12):
        df[f'mean_price_diff_0_{i + 1}'] = gbmp.apply(lambda x: x.diff(periods=i + 1))
        if i > 0:
            df[f'mean_price_diff_{i}_{i + 1}'] = gbmp.apply(lambda x: x.diff().shift(i))

    df['target'] = df.groupby('id_house')['mean_price'].shift(-1)

    df = df.merge(clusters, 'left', 'id_house')
    df['cluster'] = df['cluster'].fillna(-1).astype(int)#.map(clusters['cluster'].value_counts())

    to_predict = df['date'] == df['date'].sort_values().unique()[-1]#['target'].isna()
    df.loc[to_predict, 'target'] = model.predict(df.loc[to_predict, feature_cols + cat_cols]) + df.loc[to_predict, 'mean_price']
    return df.loc[to_predict, ['id_house', 'date_pred', 'target']]

In [ ]:
mag = sdf[pd.to_datetime(sdf['date'], format='%Y-%m') < valid_threshold].copy().sort_values(by=['id_house', 'date'])
preds = []
for i in range(N_valid_months):
    p = predict_next_month(mag, model)
    p.columns = ['id_house', 'date', 'mean_price']
    preds.append(p)
    mag = pd.concat([mag, p], axis=0)

preds = pd.concat(preds, axis=0)
valid_data = sdf[pd.to_datetime(sdf['date'], format='%Y-%m') >= valid_threshold].copy().sort_values(by=['id_house', 'date'])
merged = valid_data.merge(preds, on=['id_house', 'date'], suffixes=('_actual', '_pred'))
merged.dropna(subset=['mean_price_actual', 'mean_price_pred'], inplace=True)
raw_mae = mean_absolute_error(merged['mean_price_actual'], merged['mean_price_pred'])
print("Recursive MAE (valid):  ", raw_mae)
print("Recursive Median Error: ", (merged['mean_price_actual'] - merged['mean_price_pred']).median())

In [37]:
full_train = df.dropna(subset=['target']).sample(frac=1, random_state=42)
X_full = full_train[feature_cols + cat_cols]
y_full = full_train['target']

In [ ]:
corr_vals = X_full.corr().abs().values
print(bool(corr_vals[~np.eye(corr_vals.shape[0],dtype=bool)].max() < 0.95) == True)
X_full.corr().abs()
for i in range(corr_vals.shape[0]):
    for j in range(i + 1, corr_vals.shape[1]):
        if corr_vals[i, j] >= 0.95:
            print(X_full.columns[i], X_full.columns[j])
print(len(X_full.columns))

In [ ]:
final_model = CatBoostRegressor(
    iterations=4242,
    learning_rate=0.0005,
    depth=12,
    subsample=0.8,
    colsample_bylevel=0.9,
    l2_leaf_reg=1.0,
    random_seed=42,
    cat_features=cat_features_indices,
    loss_function='MAE',
    eval_metric='MAE',
    verbose=100
)

final_model.fit(
    X_full,
    y_full
)

In [ ]:
test['date_pred'] = test['date'].dt.to_period('M').astype(str)
test

In [ ]:
test

In [42]:
preds = []
mag = sdf.copy()
for i in range(3):
    p = predict_next_month(mag, final_model)
    preds.append(p.copy())
    p.columns = ['id_house', 'date', 'mean_price']
    mag = pd.concat([mag, p], axis=0)
preds = pd.concat(preds, axis=0)

In [ ]:
test.drop(columns=['target'], axis=1).merge(preds, 'left', on=['id_house', 'date_pred'])[60:70]

In [44]:
test = test.drop(columns=['target'], axis=1).merge(preds, 'left', on=['id_house', 'date_pred'])
# test['target'] = test.groupby('id_house')['target'].transform('mean')
test = test.groupby('id')[['target']].mean()

In [45]:
test.to_csv('submission.csv')

In [ ]:
basesub = pd.read_csv('submissionb.csv', index_col='id')
basesub

In [ ]:
from tqdm import tqdm


for i in tqdm(test.index):
    if test['target'].isna()[i]:
        test['target'][i] = basesub['target'][i]

In [48]:
test.to_csv('submission.csv')